In [1]:
# !pip3 install category_encoders
# !pip3 install scikit-learn
# !pip3 install shap

# Libraries

In [2]:
import glob
import math
import re
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from collections import Counter
from pprint import pprint

from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import shap

C:\Users\Featherine\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load Data

In [3]:
file_paths = glob.glob('Kaggle_HDB/*.csv')

df = []
for file in file_paths:
    print(file)
    df_temp = pd.read_csv(file, parse_dates=['month'])
    df.append(df_temp)

# 3. Concatenate all DataFrames into one
df_combined = pd.concat(df, ignore_index=True).reset_index(drop=True)
df_combined = df_combined.sort_values(by='month')
df_combined

Kaggle_HDB\resale-flat-prices-based-on-approval-date-1990-1999.csv
Kaggle_HDB\resale-flat-prices-based-on-approval-date-2000-feb-2012.csv
Kaggle_HDB\resale-flat-prices-based-on-registration-date-from-jan-2015-to-dec-2016.csv
Kaggle_HDB\resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv
Kaggle_HDB\resale-flat-prices-based-on-registration-date-from-mar-2012-to-dec-2014.csv


,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease
0,1990-01-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,9000.0,NaN
1010,1990-01-01,KALLANG/WHAMPOA,3 ROOM,44,BENDEMEER RD,04 TO 06,63.0,STANDARD,1981,31400.0,NaN
1009,1990-01-01,KALLANG/WHAMPOA,3 ROOM,20,ST. GEORGE'S RD,04 TO 06,67.0,NEW GENERATION,1984,66500.0,NaN
1008,1990-01-01,KALLANG/WHAMPOA,3 ROOM,14,KG ARANG RD,04 TO 06,103.0,NEW GENERATION,1984,77000.0,NaN
1007,1990-01-01,KALLANG/WHAMPOA,3 ROOM,46,OWEN RD,01 TO 03,68.0,NEW GENERATION,1982,58000.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
772720,2020-09-01,HOUGANG,4 ROOM,602,HOUGANG AVE 4,07 TO 09,84.0,Simplified,1985,345000.0,64 years
772721,2020-09-01,HOUGANG,4 ROOM,603,HOUGANG AVE 4,04 TO 06,104.0,Model A,1985,375000.0,64 years 01 month
772722,2020-09-01,HOUGANG,4 ROOM,334,HOUGANG AVE 5,04 TO 06,104.0,Model A,1983,368000.0,61 years 07 months
772715,2020-09-01,HOUGANG,4 ROOM,516,HOUGANG AVE 10,04 TO 06,104.0,Model A,1986,490000.0,64 years 11 months


# Preprocessing

In [4]:
# def round_to_multiple(x, m=5):
#     return m * round(float(x) / m)

def round_to_multiple(x, m=5):
    return int(math.ceil(float(x) / m)) * m

In [5]:
# Preprocess floor_area_sqm
# - Round price to nearest 10
df_combined['floor_area_sqm'] = df_combined['floor_area_sqm'].apply(lambda x: round_to_multiple(x, 10))

print('Count:', len(set(df_combined['floor_area_sqm'])))
# pprint(Counter(df_combined['floor_area_sqm']))

Count: 28


In [6]:
# Preprocess resale_price
# - Round price to nearest 50k
df_combined['resale_price_normalized'] = df_combined['resale_price'].apply(lambda x: int(round_to_multiple(x, 50000) / 1000))
df_combined['resale_price'] = df_combined['resale_price'].apply(lambda x: x/1000)
print('Count:', len(set(df_combined['resale_price_normalized'])))
# pprint(Counter(df_combined['resale_price_normalized']))

Count: 26


In [7]:
# Preprocess flat_model
def preprocess_flat_model(x):
    x = x.lower()
    if 'maisonette' in x:
        return 'maisonette'
    else:
        return x
    
def group_flat_model(x):
    if x in ['2-room', 'type s1', 'type s2', 'simplified']:
        return 1
    elif x in ['model a', 'standard', 'improved', 'apartment', 'new generation']:
        return 2
    elif x in ['dbss', 'premium apartment', 'model a2', 'adjoined flat']:
        return 3
    elif x in ['terrace', 'maisonette', 'multi generation']:
        return 4
    elif x in ['premium apartment loft']:
        return 5
    return x
    
df_combined['flat_model'] = df_combined['flat_model'].apply(lambda x: preprocess_flat_model(x))
df_combined['flat_model'] = df_combined['flat_model'].apply(lambda x: group_flat_model(x))

print('Count:', len(set(df_combined['flat_model'])))
# pprint(Counter(df_combined['flat_model']))

Count: 5


In [8]:
# Preprocess storey_range
def preprocess_storey_avg(x):
    x = x.lower()
    x = x.split(' to ')
    avg = (int(x[1]) + int(x[0]))/2
    return int(avg)
    
# Label encoding
def preprocess_storey_height(x):
    x = x.lower()
    x = x.split(' to ')
    avg = (int(x[1]) + int(x[0]))/2
    if avg < 5:
        return 1 #'low floor'
    elif avg >=10:
        return 3#'high floor'
    else:
        return 2 #'mid floor'
        
df_combined['storey_range_avg'] = df_combined['storey_range'].apply(lambda x: preprocess_storey_avg(x))
df_combined['storey_range_height'] = df_combined['storey_range'].apply(lambda x: preprocess_storey_height(x))
df_combined[['storey_low', 'storey_high']] = df_combined['storey_range'].str.split(' TO ', expand=True).astype(int)

print('storey_range_avg')
print('Count:', len(set(df_combined['storey_range_avg'])))
# pprint(Counter(df_combined['storey_range_avg']))

print('storey_range_height')
print('Count:', len(set(df_combined['storey_range_height'])))
# pprint(Counter(df_combined['storey_range_height']))

print('storey_low')
print('Count:', len(set(df_combined['storey_low'])))
# pprint(Counter(df_combined['storey_low']))

print('storey_high')
print('Count:', len(set(df_combined['storey_high'])))
# pprint(Counter(df_combined['storey_high']))

storey_range_avg
Count: 22
storey_range_height
Count: 3
storey_low
Count: 22
storey_high
Count: 23


In [9]:
# Preprocess lease_commence_date
def remaining_lease_to_months(s: str) -> int:
    """
    Convert strings like "64 years 01 month" or "65 years" into total months.
    """
    try:
        # Regex to capture years and months (case-insensitive)
        pattern = re.compile(r'(\d+)\s*years?(?:\s+(\d+)\s*months?)?', re.IGNORECASE)
        match = pattern.match(s.strip())

        if not match:
            raise ValueError(f"Unable to parse lease string: '{s}'")

        years = int(match.group(1))
        months = int(match.group(2)) if match.group(2) else 0
        
        return years * 12 + months
    except:
        return -1

df_combined['remaining_lease_month'] = df_combined['remaining_lease'].apply(lambda x: remaining_lease_to_months(x))
df_combined['flat_age'] = df_combined['month'].dt.year - df_combined['lease_commence_date']
df_combined['flat_age_normalized'] = df_combined['flat_age'].apply(lambda x: round_to_multiple(x, 5))

print('remaining_lease_month')
print('Count:', len(set(df_combined['remaining_lease_month'])))
# pprint(Counter(df_combined['remaining_lease_month']))

print('flat_age')
print('Count:', len(set(df_combined['flat_age'])))
# pprint(Counter(df_combined['flat_age']))

print('flat_age_normalized')
print('Count:', len(set(df_combined['flat_age_normalized'])))
# pprint(Counter(df_combined['flat_age_normalized']))

remaining_lease_month
Count: 621
flat_age
Count: 57
flat_age_normalized
Count: 12


In [10]:
# Preprocess flat_type
def preprocess_flat_type(x):
    x = x.replace('-', ' ')
    return x

def group_flat_type(x):
    flat_groups = {
        '1 ROOM': 1,
        '2 ROOM': 2,
        '3 ROOM': 3,
        '4 ROOM': 4,
        '5 ROOM': 5,
        'MULTI GENERATION': 6,
        'EXECUTIVE': 7,
    }
    
    return flat_groups[x]
    

df_combined['flat_type'] = df_combined['flat_type'].apply(lambda x: preprocess_flat_type(x))
df_combined['flat_type'] = df_combined['flat_type'].apply(lambda x: group_flat_type(x))

print('flat_type')
print('Count:', len(set(df_combined['flat_type'])))
# pprint(Counter(df_combined['flat_type']))

flat_type
Count: 7


In [11]:
# # Preprocess town
# area_to_region = {
#     'TAMPINES': 'East', 'BEDOK': 'East', 'PASIR RIS': 'East', 'CHANGI': 'East',
#     'YISHUN': 'North', 'WOODLANDS': 'North', 'SEMBAWANG': 'North', 'MANDAI': 'North',
#     'ANG MO KIO': 'North-East', 'HOUGANG': 'North-East', 'SENGKANG': 'North-East',
#     'BUKIT BATOK': 'West', 'CHOA CHU KANG': 'West', 'BUKIT PANJANG': 'West',
#     'BUKIT MERAH': 'Central', 'QUEENSTOWN': 'Central', 'CLEMENTI': 'Central',
#     'GEYLANG': 'Central', 'KALLANG/WHAMPOA': 'Central', 'MARINE PARADE': 'Central',
#     'TOA PAYOH': 'Central', 'BISHAN': 'Central', 'SERANGOON': 'Central',
#     'JURONG WEST': 'West', 'JURONG EAST': 'West', 'BUKIT TIMAH': 'Central',
#     'CENTRAL AREA': 'Central',
#     'PUNGGOL': 'North-East', 'LIM CHU KANG': 'West'
# }

# df_combined['region'] = df_combined['town'].apply(lambda x: area_to_region[x])

# print('region')
# print('Count:', len(set(df_combined['region'])))
# pprint(Counter(df_combined['region']))

In [12]:
df_combined

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,resale_price_normalized,storey_range_avg,storey_range_height,storey_low,storey_high,remaining_lease_month,flat_age,flat_age_normalized
0,1990-01-01,ANG MO KIO,1,309,ANG MO KIO AVE 1,10 TO 12,40,2,1977,9.0,NaN,50,11,3,10,12,-1,13,15
1010,1990-01-01,KALLANG/WHAMPOA,3,44,BENDEMEER RD,04 TO 06,70,2,1981,31.4,NaN,50,5,2,4,6,-1,9,10
1009,1990-01-01,KALLANG/WHAMPOA,3,20,ST. GEORGE'S RD,04 TO 06,70,2,1984,66.5,NaN,100,5,2,4,6,-1,6,10
1008,1990-01-01,KALLANG/WHAMPOA,3,14,KG ARANG RD,04 TO 06,110,2,1984,77.0,NaN,100,5,2,4,6,-1,6,10
1007,1990-01-01,KALLANG/WHAMPOA,3,46,OWEN RD,01 TO 03,70,2,1982,58.0,NaN,100,2,1,1,3,-1,8,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
772720,2020-09-01,HOUGANG,4,602,HOUGANG AVE 4,07 TO 09,90,1,1985,345.0,64 years,350,8,2,7,9,768,35,35
772721,2020-09-01,HOUGANG,4,603,HOUGANG AVE 4,04 TO 06,110,2,1985,375.0,64 years 01 month,400,5,2,4,6,769,35,35
772722,2020-09-01,HOUGANG,4,334,HOUGANG AVE 5,04 TO 06,110,2,1983,368.0,61 years 07 months,400,5,2,4,6,739,37,40
772715,2020-09-01,HOUGANG,4,516,HOUGANG AVE 10,04 TO 06,110,2,1986,490.0,64 years 11 months,500,5,2,4,6,779,34,35


In [13]:
list_columns = [
    'town',
    # 'flat_type',
    # 'resale_price'
    ]

for col in list_columns:
    print('')
    print(col)
    print('Count:', len(set(df_combined[col])))
    # pprint(Counter(df_combined[col]))


town
Count: 27


In [14]:
# list_columns = [
#     'town',
#     'flat_type',
#     # 'storey_range',
#     # 'storey_range_avg',
#     'storey_range_height',
#     'storey_low', 'storey_high',
#     'floor_area_sqm',
#     'flat_model',
#     # 'lease_commence_date', 
#     'remaining_lease',
#     # 'remaining_lease_yr',
#     'flat_age',
#     'flat_age_normalized',
#     'resale_price'
#     ]

# for col in list_columns:
#     print('')
#     print(col)
#     print('Count:', len(set(df_combined[col])))
#     pprint(Counter(df_combined[col]))

# Train/Test split and feature preprocessing

In [15]:
# Get year and month
df_combined['year'] = pd.to_datetime(df_combined['month']).dt.year
# df_combined['month'] = pd.to_datetime(df_combined['month']).dt.month

# Get cyclic encoding for month
# df_combined['month_sin'] = np.sin(2 * np.pi * df_combined['month'] / 12)
# df_combined['month_cos'] = np.cos(2 * np.pi * df_combined['month'] / 12)

df_combined.loc[df_combined['remaining_lease_month'] < 0, 'remaining_lease_month'] = np.nan
df_combined.loc[df_combined['flat_age'] < 0, 'flat_age'] = np.nan

df_train = df_combined[(df_combined['year'] >= 1990) & (df_combined['year'] <= 2018)].copy()
df_test  = df_combined[(df_combined['year'] >= 2019) & (df_combined['year'] <= 2020)].copy()

print('Combined:', len(df_combined))
print('Train   :', len(df_train))
print('Test    :', len(df_test))

Combined: 826581
Train   : 788277
Test    : 38304


In [ ]:
# Drop columns that we will not be using
list_columns_drop = [
    'block', 'street_name', 'storey_range', 'month',
    'flat_age_normalized', 'remaining_lease', 'lease_commence_date', 
    'storey_low', 'storey_high',
    'resale_price_normalized',
    # 'year', 'town', 'storey_range_avg', 
]

df_train = df_train.drop(list_columns_drop, axis=1)
df_test = df_test.drop(list_columns_drop, axis=1)

In [17]:
X_train = df_train.drop(['resale_price'], axis=1)
X_test = df_test.drop(['resale_price'], axis=1)

y_train = df_train['resale_price']
y_test = df_test['resale_price']

In [18]:
categorical_columns = ['town'] #['region']
numerical_columns = [
    'remaining_lease_month', 
    'flat_age',
    ]

# Define the parameter grid
param_grid = {
    'regressor__n_estimators': [50, 100],
    'regressor__max_depth': [10, 20]
}

# Preprocessing for numerical data: impute missing values and scale
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Preprocessing for categorical data: impute missing values and apply one-hot encoding
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine both into a ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_columns),
        ('cat', categorical_transformer, categorical_columns)
    ])

# Create the full pipeline with preprocessing and regression model
regressor = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor())
])

# # Define the scoring metrics
# scoring = ['neg_mean_squared_error', 'r2']

# # Initialize GridSearchCV
# regressor = GridSearchCV(
#     estimator=pipeline, 
#     param_grid=param_grid,
#     scoring=scoring, 
#     refit='r2', cv=3
#     )


In [ ]:
# Train the model
regressor.fit(X_train, y_train)

# Access the best parameters and scores
# print("Best parameters:", regressor.best_params_)
# print("Best R^2 score:", regressor.best_score_)

# Save the model to disk
filename = 'model/regressor_rf.sav'
pickle.dump(regressor, open(filename, 'wb')) 

In [ ]:
y_pred = regressor.predict(X_test)

# Evaluate the model
print("Training score:", regressor.score(X_train, y_train))
print("Test score    :", regressor.score(X_test, y_test))

mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print('')
print("MAE :", mae)
print("RMSE:", rmse)
print("R2  :", r2)

# Save test predictions
results = X_test.copy()
results['label'] = y_test
results['predict'] = y_pred
results.to_csv("results/test_predictions.csv", index=False)

In [ ]:
# Save results
results = f'R2: {r2} - MAE: {mae} - RMSE: {rmse}'

plt.figure(figsize=(5, 5))
plt.scatter(y_test, y_pred, alpha=0.6, edgecolors="w", linewidth=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], "k--", lw=2)
plt.xlabel("Actual")
plt.ylabel("Predicted")
plt.title("Actual vs Predicted Housing Price")
plt.show()
plt.savefig('results/actual_vs_predict.png')